In [1]:
import sqlite3
import pandas as pd


db_path = "db_minitest.sqlite3" 
conn = sqlite3.connect(db_path)

In [2]:
def run_query(query):
    return pd.read_sql_query(query, conn)

In [3]:

query1 = """
SELECT u.name, u.email, p.phone, p.address, u.created_at
FROM users u
JOIN profiles p ON u.user_id = p.user_id
WHERE u.age > 30
  AND strftime('%Y', u.created_at) = strftime('%Y', 'now')
ORDER BY u.created_at ASC
LIMIT 5;

"""
print("Query 1: Users older than 30 created this year, ordered by creation date:")
display(run_query(query1))

Query 1: Users older than 30 created this year, ordered by creation date:


,name,email,phone,address,created_at
0,Diana,diana@example.com,555-3456,321 Pine St,2025-03-15
1,Edward,edward@example.com,555-7890,654 Cedar Ave,2025-04-10
2,Hannah,hannah@example.com,555-0123,135 Willow Dr,2025-06-01


In [4]:
query2 = """
SELECT u.user_id, u.name, SUM(o.amount) AS total_spent
FROM users u
JOIN orders o ON u.user_id = o.user_id
GROUP BY u.user_id, u.name
HAVING total_spent > 700;

"""
print("Query 2: Users who spent more than $700:")
display(run_query(query2))

Query 2: Users who spent more than $700:


,user_id,name,total_spent
0,1,Alice,1225.5


In [5]:
query3= """
SELECT DISTINCT p.address
FROM profiles p
JOIN users u ON p.user_id = u.user_id
JOIN orders o ON u.user_id = o.user_id
GROUP BY p.address
HAVING COUNT(o.order_id) >= 2;

"""
print("Query 3: Addresses with at least 2 orders:")
display(run_query(query3))

Query 3: Addresses with at least 2 orders:


,address
0,123 Main St
1,135 Willow Dr
2,321 Pine St
3,654 Cedar Ave
4,987 Birch Rd


In [6]:
query4 = """
SELECT
  CASE
    WHEN age < 25 THEN 'Jovenes'
    WHEN age BETWEEN 25 AND 40 THEN 'Adultos'
    ELSE 'Seniors'
  END AS grupo_edad,
  COUNT(*) AS total_usuarios
FROM users
GROUP BY grupo_edad;

"""
print("Query 4: Users categorized by age group:")
display(run_query(query4))

Query 4: Users categorized by age group:


,grupo_edad,total_usuarios
0,Adultos,3
1,Jovenes,2
2,Seniors,3


In [7]:
query5 = """
SELECT u.user_id, u.name, u.age, u.email, p.phone, p.address,
       o.order_id, o.product, o.amount, o.order_date
FROM users u
JOIN profiles p ON u.user_id = p.user_id
JOIN orders o ON u.user_id = o.user_id
ORDER BY o.order_date DESC;
"""
print("Query 5: All user details with their latest order:")
display(run_query(query5))

Query 5: All user details with their latest order:


,user_id,name,age,email,phone,address,order_id,product,amount,order_date
0,8,Hannah,42,hannah@example.com,555-0123,135 Willow Dr,1015,Charger,20.0,2025-06-06
1,8,Hannah,42,hannah@example.com,555-0123,135 Willow Dr,1014,Smartwatch,250.0,2025-06-05
2,6,Fiona,27,fiona@example.com,555-2345,987 Birch Rd,1012,Filter,30.0,2025-05-17
3,6,Fiona,27,fiona@example.com,555-2345,987 Birch Rd,1011,Bag,50.0,2025-05-16
4,6,Fiona,27,fiona@example.com,555-2345,987 Birch Rd,1010,Lens,200.0,2025-05-15
5,5,Edward,55,edward@example.com,555-7890,654 Cedar Ave,1009,Tripod,80.0,2025-04-16
6,5,Edward,55,edward@example.com,555-7890,654 Cedar Ave,1008,Camera,500.0,2025-04-15
7,4,Diana,35,diana@example.com,555-3456,321 Pine St,1007,Webcam,55.0,2025-04-01
8,4,Diana,35,diana@example.com,555-3456,321 Pine St,1006,Headphones,85.0,2025-03-25
9,4,Diana,35,diana@example.com,555-3456,321 Pine St,1005,Tablet,450.0,2025-03-20


VAN IA


In [10]:
from vanna.remote import VannaDefault
import os

In [11]:

api_key = os.getenv("API_KEY")
vanna_model_name =  os.getenv("VANNA_MODEL_NAME")
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

ENTRENAR MODELO

In [12]:
vn.train(
    ddl="""
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    email TEXT NOT NULL UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);"""
)

Adding ddl: 
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    email TEXT NOT NULL UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);


'16993643-ddl'

In [13]:
vn.train(
    ddl="""
CREATE TABLE profiles (
    user_id INTEGER PRIMARY KEY,
    address TEXT NOT NULL,
    phone TEXT NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);"""
)

Adding ddl: 
CREATE TABLE profiles (
    user_id INTEGER PRIMARY KEY,
    address TEXT NOT NULL,
    phone TEXT NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);


'16993648-ddl'

In [14]:
vn.train(
    ddl="""
    CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,   
    product TEXT NOT NULL,
    amount REAL NOT NULL,
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);"""
)

Adding ddl: 
    CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,   
    product TEXT NOT NULL,
    amount REAL NOT NULL,
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);


'16993673-ddl'

In [15]:
vn.train(
    documentation="El usuario puede usar el nombre de pedido en lugar de orden."
)

Adding documentation....


'3443501-doc'

In [16]:
query = vn.generate_sql(
    "Quien es el usuario que hizo el ultimo pedido?"
)

SQL Prompt: [{'role': 'system', 'content': "You are a SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\n    CREATE TABLE orders (\n    order_id INTEGER PRIMARY KEY,\n    user_id INTEGER NOT NULL,   \n    product TEXT NOT NULL,\n    amount REAL NOT NULL,\n    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n    FOREIGN KEY (user_id) REFERENCES users(user_id)\n);\n\n\nCREATE TABLE users (\n    user_id INTEGER PRIMARY KEY,\n    name TEXT NOT NULL,\n    age INTEGER NOT NULL,\n    email TEXT NOT NULL UNIQUE,\n    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n);\n\n\nCREATE TABLE profiles (\n    user_id INTEGER PRIMARY KEY,\n    address TEXT NOT NULL,\n    phone TEXT NOT NULL,\n    FOREIGN KEY (user_id) REFERENCES users(user_id)\n);\n\n\n===Additional Context \n\nEl usuario puede usar el nombre de pedido en lugar de orden.\n\n===Response 

In [18]:
pd.read_sql_query(query, conn)

,name
0,Hannah


In [19]:
query2 = vn.generate_sql(
    "Cuales son los datos de el usuario con mas pedidos?")

SQL Prompt: [{'role': 'system', 'content': "You are a SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\n    CREATE TABLE orders (\n    order_id INTEGER PRIMARY KEY,\n    user_id INTEGER NOT NULL,   \n    product TEXT NOT NULL,\n    amount REAL NOT NULL,\n    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n    FOREIGN KEY (user_id) REFERENCES users(user_id)\n);\n\n\nCREATE TABLE users (\n    user_id INTEGER PRIMARY KEY,\n    name TEXT NOT NULL,\n    age INTEGER NOT NULL,\n    email TEXT NOT NULL UNIQUE,\n    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n);\n\n\nCREATE TABLE profiles (\n    user_id INTEGER PRIMARY KEY,\n    address TEXT NOT NULL,\n    phone TEXT NOT NULL,\n    FOREIGN KEY (user_id) REFERENCES users(user_id)\n);\n\n\n===Additional Context \n\nEl usuario puede usar el nombre de pedido en lugar de orden.\n\n===Response 

In [20]:
pd.read_sql_query(query2, conn)

,user_id,name,age,email,address,phone
0,6,Fiona,27,fiona@example.com,987 Birch Rd,555-2345
